In [ ]:
# Import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# Get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load and prepare the data
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]
train_labels = train_df["type"].values
test_labels = test_df["type"].values

In [ ]:
# Initialize and fit the tokenizer
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["msg"])

In [ ]:
# Convert text to sequences and add padding
train_sequences = tokenizer.texts_to_sequences(train_df["msg"])
test_sequences = tokenizer.texts_to_sequences(test_df["msg"])
maxlen = max(train_df["msg"].apply(lambda x: len(x.split())))
train_sequences = pad_sequences(train_sequences, maxlen=maxlen, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=maxlen, padding='post')

In [ ]:
# Create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 50, input_length=maxlen))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.003), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(
    train_sequences,
    train_labels,
    validation_data=(test_sequences, test_labels),
    epochs=35,
    verbose="auto"
)

In [ ]:
# Evaluate the model
evaluation_metrics = model.evaluate(test_sequences, test_labels, verbose="auto")
print('Evaluation Results -> Loss:', evaluation_metrics[0], ', Accuracy:', evaluation_metrics[1])

In [ ]:
# Function to return prediction
def predict_message(pred_text):

    if not isinstance(pred_text, str) or len(pred_text.strip()) == 0:
        raise ValueError("Input must be a non-empty string.")

    seq = tokenizer.texts_to_sequences([pred_text])
    seq = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='pre')

    prediction_score = model.predict(seq)[0, 0]
    label = "ham" if prediction_score < 0.5 else "spam"

    return [prediction_score, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
